In [9]:
#model = trainNAME(X, Y, ...{other params}...), predictNAME(model, X), где {other params} 
#– это параметры определяемые студентом, включающие все используемые в модели константы, 
#model – обученная модель, возвращаемая первой функцией. Функция предсказания должна возвращать вероятность класса 1.
import pandas as pd
import numpy as np
data=pd.read_csv("./exported_data.csv",sep=',', decimal=",") # Считываем данные из файла


eps = np.finfo(float).eps
from numpy import log2 as log

data.head()  
  

,Bought_Or_Not,Recency,Frequency,Monetary_Value,Mean_Quantity,ActivationDate,Age,SubscribedEmail,SubscribedPush,Male
0,0.0,160.0,8.142857142857142,251.78947368421052,1.0350877192982457,735599,24.0,False,True,0
1,0.0,720.0,4.857142857142857,279.6470588235294,1.0,735599,46.0,True,True,1
2,1.0,15.0,4.714285714285714,398.7348484848485,1.6666666666666667,735599,27.0,True,True,0
3,0.0,177.0,14.142857142857142,262.7373737373738,1.1414141414141414,735599,54.0,True,True,0
4,1.0,115.0,3.0,393.7142857142857,1.8095238095238095,735599,48.0,False,False,1


In [10]:
def find_entropy(data): #найдем общую энтропию
    Class = data.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0 #переменная для общей энтропии
    values = data[Class].unique() #предсказываемая переменная
    for value in values:  #идем по каждой константе и считаем
        fraction = data[Class].value_counts()[value]/len(data[Class]) #вероятность 
        entropy += -fraction*np.log2(fraction) #сама энтропия
    return entropy


# посчитаем энтропию для каждого аттрибута
def find_entropy_attribute(data,attribute):
  Class = data.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = data[Class].unique()  #This gives all 'Yes' and 'No'
  variables = data[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(data[attribute][data[attribute]==variable][data[Class] ==target_variable])
          den = len(data[attribute][data[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(data)
      entropy2 += -fraction2*entropy
  return abs(entropy2)

In [11]:
# найдем прирост информации
def find_winner(data):
    Entropy_att = []
    IG = []
    for key in data.keys()[:-1]:
#         Entropy_att.append(find_entropy_attribute(data,key))
        IG.append(find_entropy(data)-find_entropy_attribute(data,key))
    return data.keys()[:-1][np.argmax(IG)]

In [ ]:
# построим дерево решений
def buildTree(data,tree=None): 
    Class = data.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(data)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(data[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(data,node,value)
        clValue,counts = np.unique(subtable['Bought_Or_Not'],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree




In [ ]:
# нарисуем дерево решений
tree=buildTree(data)
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(25,10))
a = plot_tree(tree, 
              feature_names=data.feature_names, 
              class_names=data.target_names, 
              filled=True, 
              rounded=True, 
              fontsize=14)
print(a)

NameError: name 'a' is not defined